In [1]:
import pandas as pd
CompanyData = pd.read_csv("C:\\Users\\alfre_g2qn6y7\\OneDrive\\Documents\\Company Financial Data.csv")
CompanyData

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding
0,0,AAL,12/31/2012,3.068000e+09,-222000000,-1961000000,23,-1888000000,4.695000e+09,53.0,...,7.072000e+09,9011000000,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,2012.0,-5.60,3.350000e+08
1,1,AAL,12/31/2013,4.975000e+09,-93000000,-2723000000,67,-3114000000,1.059200e+10,75.0,...,1.432300e+10,13806000000,-2.731000e+09,4.500900e+10,4.227800e+10,2.674300e+10,0.0,2013.0,-11.25,1.630222e+08
2,2,AAL,12/31/2014,4.668000e+09,-160000000,-150000000,143,-5311000000,1.513500e+10,60.0,...,1.175000e+10,13404000000,2.021000e+09,4.120400e+10,4.322500e+10,4.265000e+10,0.0,2014.0,4.02,7.169154e+08
3,3,AAL,12/31/2015,5.102000e+09,352000000,-708000000,135,-6151000000,1.159100e+10,51.0,...,9.985000e+09,13605000000,5.635000e+09,4.278000e+10,4.841500e+10,4.099000e+10,0.0,2015.0,11.39,6.681299e+08
4,4,AAP,12/29/2012,2.409453e+09,-89482000,600000,32,-271182000,5.202150e+08,23.0,...,3.184200e+09,2559638000,1.210694e+09,3.403120e+09,4.613814e+09,6.205003e+09,-27095000.0,2012.0,5.29,7.328355e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1776,1776,ZION,12/31/2015,0.000000e+00,0,-2530000,4,-157361000,0.000000e+00,NaN,...,0.000000e+00,0,7.507519e+09,5.216201e+10,5.966952e+10,2.210591e+09,0.0,2015.0,1.20,2.578925e+08
1777,1777,ZTS,12/31/2013,1.381000e+09,-99000000,9000000,54,-184000000,8.780000e+08,43.0,...,3.357000e+09,1415000000,9.400000e+08,5.618000e+09,6.558000e+09,4.561000e+09,0.0,2013.0,1.01,4.990099e+08
1778,1778,ZTS,12/31/2014,1.071000e+09,69000000,-7000000,44,-180000000,9.580000e+08,81.0,...,3.465000e+09,1086000000,1.311000e+09,5.277000e+09,6.588000e+09,4.785000e+09,0.0,2014.0,1.16,5.025862e+08
1779,1779,ZTS,12/31/2015,1.313000e+09,-58000000,-81000000,32,-224000000,1.012000e+09,65.0,...,3.830000e+09,1781000000,1.068000e+09,6.845000e+09,7.913000e+09,4.765000e+09,-203000000.0,2015.0,0.68,4.985294e+08


In [3]:
# Calculate quartiles for the net income
quartiles = CompanyData["Net Income"].quantile([0.25, 0.75])

# Define the categories based on quartiles
def categorize_profit(net_income):
    if net_income >= quartiles[0.75]:
        return 'High Profit'
    elif net_income <= quartiles[0.25]:
        return 'Low Profit'
    else:
        return 'Medium Profit'

# Apply the categorization
CompanyData['Profit Category'] = CompanyData["Net Income"].apply(categorize_profit)


In [5]:
CompanyData["Profit Category"]

0          Low Profit
1          Low Profit
2         High Profit
3         High Profit
4       Medium Profit
            ...      
1776       Low Profit
1777    Medium Profit
1778    Medium Profit
1779       Low Profit
1780    Medium Profit
Name: Profit Category, Length: 1781, dtype: object

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
X = CompanyData[["Total Revenue","Current Ratio","Capital Expenditures","Total Assets"]]
y = CompanyData["Profit Category"]
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier()
model.fit(X_train,y_train)
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test,predictions)
accuracy

0.7563025210084033

In [13]:
report = classification_report(y_test,predictions)
print("Classification Report:\n", report)

Classification Report:
                precision    recall  f1-score   support

  High Profit       0.78      0.80      0.79        81
   Low Profit       0.67      0.61      0.64        82
Medium Profit       0.78      0.80      0.79       194

     accuracy                           0.76       357
    macro avg       0.74      0.74      0.74       357
 weighted avg       0.75      0.76      0.75       357



In [15]:
conf_matrix = confusion_matrix(y_test, predictions)
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[ 65   0  16]
 [  4  50  28]
 [ 14  25 155]]


In [17]:
importances = model.feature_importances_
feature_names = X.columns

# Create a DataFrame for visualization
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the most important features
print(feature_importance_df)

                Feature  Importance
0         Total Revenue    0.342341
3          Total Assets    0.311679
2  Capital Expenditures    0.177086
1         Current Ratio    0.168894
